In [22]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import os
path = 'individual-datasets/drought-data/'

In [23]:
counties = os.listdir(path)
remove = []
for i in range(len(counties)):
    if '.csv' in counties[i]:
        counties[i] = counties[i][:-4].title()
    else:
        remove.append(counties[i])

for r in remove:
    counties.remove(r)

In [24]:
files = os.listdir(path)
drought_dfs = {}
for f in files:
    if '.csv' in f:
        drought_dfs[f[:-4].title()] = pd.read_csv(path+f)

In [25]:
final_df = pd.DataFrame()
for county, df in drought_dfs.items():
    df = df[['Week', 'DSCI']]
    df.Week = pd.to_datetime(df.Week)
    df = pd.DataFrame(df.groupby(df.Week.dt.year)['DSCI'])
    df[1] = df[1].apply(lambda x: x.mean())
    df.columns = ['Year', county]
    df = df.set_index('Year')
    if len(final_df) > 0:
        final_df = final_df.join(df, on=['Year'], how='left')
    else:
        final_df = df.copy()
final_df = pd.melt(final_df.reset_index(), id_vars = ['Year'], value_vars = counties)
final_df = final_df.rename({'value': 'DSCI', 'variable':'County'}, axis=1)
final_df

/var/folders/49/b4fjywl16vqb_14cy97qv8s40000gn/T/ipykernel_53013/4218427996.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Week = pd.to_datetime(df.Week)
/var/folders/49/b4fjywl16vqb_14cy97qv8s40000gn/T/ipykernel_53013/4218427996.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Week = pd.to_datetime(df.Week)
/var/folders/49/b4fjywl16vqb_14cy97qv8s40000gn/T/ipykernel_53013/4218427996.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

,Year,County,DSCI
0,2000,Franklin,59.461538
1,2001,Franklin,125.730769
2,2002,Franklin,307.433962
3,2003,Franklin,0.000000
4,2004,Franklin,0.000000
...,...,...,...
2370,2020,Isle Of Wight,0.826923
2371,2021,Isle Of Wight,44.211538
2372,2022,Isle Of Wight,24.057692
2373,2023,Isle Of Wight,26.461538


In [26]:
fips_map = {
    '0001': 'Accomack',
    '0003': 'Albemarle',
    '0510': 'Alexandria City',
    '0005': 'Alleghany',
    '0007': 'Amelia',
    '0009': 'Amherst',
    '0011': 'Appomattox',
    '0013': 'Arlington',
    '0015': 'Augusta',
    '0017': 'Bath',
    '0019': 'Bedford',
    '0021': 'Bland',
    '0023': 'Botetourt',
    '0520': 'Bristol City',
    '0025': 'Brunswick',
    '0027': 'Buchanan',
    '0029': 'Buckingham',
    '0530': 'Buena Vista City',
    '0031': 'Campbell',
    '0033': 'Caroline',
    '0035': 'Carroll',
    '0036': 'Charles City',
    '0037': 'Charlotte',
    '0540': 'Charlottesville City',
    '0550': 'Chesapeake City',
    '0041': 'Chesterfield',
    '0043': 'Clarke',
    '0570': 'Colonial Heights City',
    '0580': 'Covington City',
    '0045': 'Craig',
    '0047': 'Culpeper',
    '0049': 'Cumberland',
    '0590': 'Danville City',
    '0051': 'Dickenson',
    '0053': 'Dinwiddie',
    '0057': 'Essex',
    '0059': 'Fairfax',
    '0061': 'Fauquier',
    '0063': 'Floyd',
    '0065': 'Fluvanna',
    '0067': 'Franklin',
    '0620': 'Franklin City',
    '0069': 'Frederick',
    '0630': 'Fredericksburg City',
    '0640': 'Galax City',
    '0071': 'Giles',
    '0073': 'Gloucester',
    '0075': 'Goochland',
    '0077': 'Grayson',
    '0079': 'Greene',
    '0081': 'Greensville',
    '0083': 'Halifax',
    '0650': 'Hampton City',
    '0085': 'Hanover',
    '0660': 'Harrisonburg City',
    '0087': 'Henrico',
    '0089': 'Henry',
    '0091': 'Highland',
    '0670': 'Hopewell City',
    '0093': 'Isle Of Wight',
    '0095': 'James City',
    '0097': 'King And Queen',
    '0099': 'King George',
    '0101': 'King William',
    '0103': 'Lancaster',
    '0105': 'Lee',
    '0678': 'Lexington City',
    '0109': 'Louisa',
    '0107': 'Loudoun',
    '0111': 'Lunenburg',
    '0113': 'Madison',
    '0683': 'Manassas City',
    '0685': 'Manassas Park City',
    '0690': 'Martinsville City',
    '0115': 'Mathews',
    '0117': 'Mecklenburg',
    '0119': 'Middlesex',
    '0121': 'Montgomery',
    '0125': 'Nelson',
    '0127': 'New Kent',
    '0700': 'Newport News City',
    '0710': 'Norfolk City',
    '0131': 'Northampton',
    '0133': 'Northumberland',
    '0720': 'Norton City',
    '0135': 'Nottoway',
    '0137': 'Orange',
    '0139': 'Page',
    '0141': 'Patrick',
    '0730': 'Petersburg City',
    '0143': 'Pittsylvania',
    '0740': 'Portsmouth City',
    '0145': 'Powhatan',
    '0147': 'Prince Edward',
    '0149': 'Prince George',
    '0153': 'Prince William',
    '0155': 'Pulaski',
    '0750': 'Radford City',
    '0157': 'Rappahannock',
    '0159': 'Richmond',
    '0760': 'Richmond City',
    '0161': 'Roanoke',
    '0770': 'Roanoke City',
    '0163': 'Rockbridge',
    '0165': 'Rockingham',
    '0167': 'Russell',
    '0169': 'Scott',
    '0171': 'Shenandoah',
    '0173': 'Smyth',
    '0175': 'Southampton',
    '0177': 'Spotsylvania',
    '0179': 'Stafford',
    '0790': 'Staunton City',
    '0800': 'Suffolk City',
    '0181': 'Surry',
    '0183': 'Sussex',
    '0185': 'Tazewell',
    '0810': 'Virginia Beach City',
    '0187': 'Warren',
    '0191': 'Washington',
    '0820': 'Waynesboro City',
    '0193': 'Westmoreland',
    '0830': 'Williamsburg City',
    '0840': 'Winchester City',
    '0195': 'Wise',
    '0197': 'Wythe',
    '0199': 'York'
}

fips_map = {v: k for k, v in fips_map.items()}

In [27]:
final_df['County'] = ['51'+fips_map[county][-3:] for county in final_df.County]

In [28]:
len(counties)

95

In [31]:
final_df

,Year,County,DSCI
0,2000,51067,59.461538
1,2001,51067,125.730769
2,2002,51067,307.433962
3,2003,51067,0.000000
4,2004,51067,0.000000
...,...,...,...
2370,2020,51093,0.826923
2371,2021,51093,44.211538
2372,2022,51093,24.057692
2373,2023,51093,26.461538
